In [1]:
using Cxx

In [2]:
# The EIGEN_DONT_ALIGN_STATICALLY is needed because Julia does not respect the alignment constraints imposed by Eigen.
cxx"""
       #define EIGEN_DONT_ALIGN_STATICALLY
       #include "../eigen3/Eigen/Dense"
"""

true

In [8]:
# Wrapper for Eigen matrix
struct EMatrix{T,Rows,Cols,EigenT} <: AbstractMatrix{T}
    eigenmat::EigenT

    function EMatrix{T,Rows,Cols}() where {T,Rows,Cols}
        emat = icxx"Eigen::Matrix<$T,$(Val{Rows}),$(Val{Cols})>();"
        return new{T,Rows,Cols,typeof(emat)}(emat)
    end

    EMatrix{T,Rows,Cols}(emat::EigenT) where {T,Rows,Cols,EigenT} = new{T,Rows,Cols,EigenT}(emat)
end

In [9]:
# Implement the array interface
Base.IndexStyle(::Type{EMatrix{T,Rows,Cols,EigenT}}) where {T,Rows,Cols,EigenT} = IndexCartesian()
Base.size(a::EMatrix{T,Rows,Cols,EigenT}) where {T,Rows,Cols,EigenT} = (Rows,Cols)
Base.getindex(a::EMatrix{T,Rows,Cols,EigenT}, i::Int, j::Int) where {T,Rows,Cols,EigenT} =
    T(icxx"$(a.eigenmat)($(i-1),$(j-1));")
Base.setindex!(a::EMatrix, v, i::Int, j::Int) = icxx"$(a.eigenmat)($(i-1),$(j-1)) = $v;"

In [17]:
# Implement math ops here
function Base.:(+)(a::EMatrix{T,Rows,Cols,EigenT1}, b::EMatrix{T,Rows,Cols,EigenT2}) where {T,Rows,Cols,EigenT1,EigenT2}
    return EMatrix{T,Rows,Cols}(icxx"$(a.eigenmat) + $(b.eigenmat);")
end

In [29]:
# Uninitialized 4 × 4 matrix
m1 = EMatrix{Float64,4,4}()

4×4 EMatrix{Float64,4,4,Cxx.CppValue{Cxx.CxxQualType{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("Eigen::Matrix")},Tuple{Float64,Val{4},Val{4},Val{0},Val{4},Val{4}}},(false, false, false)},128}}:
 2.21988e-314  2.21988e-314  2.21988e-314  2.21988e-314
 2.21988e-314  2.21988e-314  2.21988e-314  2.21988e-314
 2.21988e-314  2.21988e-314  2.21988e-314  2.21988e-314
 2.21988e-314  2.21988e-314  2.21988e-314  2.21988e-314

In [30]:
# This works because we implemented the AbstractArray interface
rand!(m1)

4×4 EMatrix{Float64,4,4,Cxx.CppValue{Cxx.CxxQualType{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("Eigen::Matrix")},Tuple{Float64,Val{4},Val{4},Val{0},Val{4},Val{4}}},(false, false, false)},128}}:
 0.461492  0.266991   0.0336359  0.972871
 0.957333  0.299609   0.228106   0.450773
 0.499779  0.0275278  0.119247   0.729397
 0.605713  0.405994   0.754813   0.145856

In [31]:
m2 = EMatrix{Float64,4,4}()
fill!(m2,1)

4×4 EMatrix{Float64,4,4,Cxx.CppValue{Cxx.CxxQualType{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("Eigen::Matrix")},Tuple{Float64,Val{4},Val{4},Val{0},Val{4},Val{4}}},(false, false, false)},128}}:
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0

In [32]:
# Summing keeps the internal expression template (Eigen::CwiseBinaryOp here)
m1 + m2

4×4 EMatrix{Float64,4,4,Cxx.CppValue{Cxx.CxxQualType{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("Eigen::CwiseBinaryOp")},Tuple{Cxx.CxxQualType{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("Eigen::internal::scalar_sum_op")},Tuple{Float64,Float64}},(false, false, false)},Cxx.CxxQualType{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("Eigen::Matrix")},Tuple{Float64,Val{4},Val{4},Val{0},Val{4},Val{4}}},(true, false, false)},Cxx.CxxQualType{Cxx.CppTemplate{Cxx.CppBaseType{Symbol("Eigen::Matrix")},Tuple{Float64,Val{4},Val{4},Val{0},Val{4},Val{4}}},(true, false, false)}}},(true, false, false)},24}}:
 1.46149  1.26699  1.03364  1.97287
 1.95733  1.29961  1.22811  1.45077
 1.49978  1.02753  1.11925  1.7294 
 1.60571  1.40599  1.75481  1.14586